# TI Calculation


## Imports

In [1]:
#for analysis
from matplotlib import pyplot as plt
%matplotlib inline

import os
import numpy as np
import pygromos
from pygromos.files.gromos_system import Gromos_System
from pygromos.simulations.hpc_queuing.submission_systems.local import LOCAL as subSystem
from pygromos.files.blocks.imd_blocks import PERTURBATION, WRITETRAJ

from pygromos.utils import bash

## Input files

In [2]:
root_dir = os.path.abspath(os.path.dirname(pygromos.__file__)+"/../examples/example_files/TI_Calculation")
root_in_dir = root_dir+"/TI_input"
cnf_path = root_in_dir+"/M030_6KET.cnf"
top_path = root_in_dir + "/M030_6KET.top"
disres_path = root_in_dir+"/M030_6KET.disres"


sys_name = "M030_to_6KET"
lam = 0

project_dir = bash.make_folder(root_dir+"/"+sys_name)
input_dir = bash.make_folder(project_dir+"/input")

## Vacuum Simulation

### Direction A->B

#### Setup:


##### Build pertubation file


In [3]:
from pygromos.files.topology.ptp import Pertubation_topology
from pygromos.files.blocks.topology_blocks import pertubation_lam_state, atom_lam_pertubation_state, PERTATOMPARAM, TITLE


#External imd_changes:
grom_system = Gromos_System(in_cnf_path=cnf_path, in_top_path=top_path,
                            in_disres_path=disres_path,
                            system_name=sys_name, work_folder=input_dir)


#Build up lambda - States
pert_atoms=[]
for atom_line in grom_system.top.SOLUTEATOM:
    states = {}
    phys_state = pertubation_lam_state(IAC=atom_line.IAC, MASS=atom_line.MASS, CHARGE=atom_line.CG)
    states = {atom_line.MRES: phys_state }

    pert_atom = atom_lam_pertubation_state(atom_line.ATNM,RES=atom_line.MRES,NAME=atom_line.PANM, STATES=states)
    pert_atoms.append(pert_atom)
pert_atom_block = PERTATOMPARAM(pert_atoms)

# Generate ptp file
grom_system.ptp = Pertubation_topology(in_value = None)
grom_system.ptp.PERTATOMPARAM = pert_atom_block
grom_system.ptp.TITLE = TITLE("Automatic generated pertubation file. ")

grom_system.ptp

TITLE
Automatic generated pertubation file. 
	>>> Generated with PyGromosTools (riniker group) <<<
END
PERTATOMPARAM
# NJLA  	 NPTB =  	 2 	 
 	 26
# state_identifiers
#  	 state1 	 state2
#    NR   RES  NAME  IAC1mass1   CHARGE1  IAC2mass2   CHARGE2    ALPHLJ     ALPHCRF   
    1     1    C9    16 15.035    0.05200    22 15.035    0.00000    1.00000    1.00000
    2     1    C6    64 12.011    0.21300    22 12.011    0.00000    1.00000    1.00000
    3     1    C2    64 12.011   -0.43400    22 12.011    0.00000    1.00000    1.00000
    4     1    H2    20 1.008    0.16900    22 1.008    0.00000    1.00000    1.00000
    5     1    C4    64 12.011    0.21300    22 12.011    0.00000    1.00000    1.00000
    6     1    C7    16 15.035    0.05200    22 15.035    0.00000    1.00000    1.00000
    7     1    C1    64 12.011   -0.43400    22 12.011    0.00000    1.00000    1.00000
    8     1    H1    20 1.008    0.16900    22 1.008    0.00000    1.00000    1.00000
    9     1    C5    64 

In [4]:
##Write out all generated files
grom_system.rebase_files()

##save Input System
grom_system.save(root_dir+"/initial_startSys.obj")
grom_system

/home/runner/work/PyGromosTools/PyGromosTools/pygromos/files/gromos_system/gromos_system.py:869: UserWarning: Did not change file path as its only promised None
  warnings.warn("Did not change file path as its only promised " + str(file_obj.path))


File imd is empty , can not be written!


PicklingError: Can't pickle <class 'pygromos.files.blocks.topology_blocks.pertubationLamState'>: attribute lookup pertubationLamState on pygromos.files.blocks.topology_blocks failed

### RUN Emin

In [5]:
# PREPARE EMIN
## IMPORT
from pygromos.data.simulation_parameters_templates import template_emin_vac
from pygromos.simulations.modules.preset_simulation_modules import emin

grom_system = Gromos_System.load(root_dir+"/initial_startSys.obj")
step_name  = "a_emin" #also the dir_name, out prefix etc.
grom_system.imd = template_emin_vac #read template imd

#Pertubation for molecules to sim params
pert_block  =  PERTURBATION(NTG=1, NRDGL=0, RLAM=lam, DLAMT=0,
                            ALPHC=0.5, ALPHLJ=0.5, NLAM=2, NSCALE=0)
grom_system.imd.add_block(block=pert_block)

#save checkpoint system
step_dir = project_dir+"/"+step_name


EOFError: Ran out of input

In [6]:
#EXECUTE EMIN
emin_gromos_system, jobID = emin(in_gromos_system=grom_system, project_dir=project_dir,
                          step_name=step_name, submission_system=subSystem())

emin_gromos_system.save(step_dir+"/emin_out.obj")
emin_gromos_system

NameError: name 'step_name' is not defined

## RUN Test SD EQ

In [7]:
from pygromos.data.simulation_parameters_templates import template_sd
from pygromos.simulations.modules.preset_simulation_modules import sd

grom_system = Gromos_System.load(root_dir+"/M030_to_6KET/a_emin/emin_out.obj")
step_name  = "b_vacuum_sd"
grom_system.imd = template_sd

#Pertubation
pert_block  =  PERTURBATION(NTG=1, NRDGL=0, RLAM=lam, DLAMT=0,
                            ALPHC=0.5, ALPHLJ=0.5, NLAM=2, NSCALE=0)
grom_system.imd.add_block(block=pert_block)

#write out trajs:
write_traj = WRITETRAJ(NTWX=100, NTWE=100)
grom_system.imd.add_block(block=write_traj)
step_dir = project_dir+"/"+step_name

#further mods:
grom_system.imd.CONSTRAINT.NTC = 3
grom_system.imd.FORCE.BONDS = 0

grom_system.imd.COVALENTFORM.NTBAH = 0
grom_system.imd.COVALENTFORM.NTBDN = 0
grom_system.imd.COVALENTFORM.NTBBH = 0

grom_system.imd.STEP.NSTLIM = 30000




FileNotFoundError: [Errno 2] No such file or directory: '/home/runner/work/PyGromosTools/PyGromosTools/examples/example_files/TI_Calculation/M030_to_6KET/a_emin/emin_out.obj'

In [8]:
sd_gromos_system, jobID  = sd(in_gromos_system=grom_system, project_dir=project_dir,
                              step_name=step_name, submission_system=subSystem(), in_imd_path=None,
                      equilibration_runs=1, simulation_runs=1)
sd_gromos_system.save(step_dir+"/sd_out_system.obj")
sd_gromos_system

NameError: name 'step_name' is not defined

## Further Analysis:
Transform the trc into a pdb traj to check it out with pymol :!)

In [9]:
#final analysis dir:
from pygromos.utils import bash

out_ana = project_dir+"/c_ana"
if(not os.path.exists(out_ana)):
    bash.make_folder(out_ana)

In [10]:
project_dir

'/home/runner/work/PyGromosTools/PyGromosTools/examples/example_files/TI_Calculation/M030_to_6KET'

In [11]:
from pygromos.files.coord import Cnf
in_path=project_dir+"/a_emin/analysis/data/a_emin.cnf"
cnf_file = Cnf(in_path)
cnf_file.write_pdb(in_path.replace("cnf", "pdb"))


OSError: [Errno Could not find File: ] /home/runner/work/PyGromosTools/PyGromosTools/examples/example_files/TI_Calculation/M030_to_6KET/a_emin/analysis/data/a_emin.cnf

In [12]:
cnf_file.visualize()

NameError: name 'cnf_file' is not defined

In [13]:
from pygromos.files.trajectory.trc import Trc

in_path=project_dir+"/b_vacuum_sd/analysis/data/b_vacuum_sd.trc.h5"

trc = Trc(input_value=in_path)
trc.write_pdb(out_ana+"/sd_traj.pdb", cnf_file=cnf_path)#grom_system.cnf.path)
trc

TypeError: __init__() got an unexpected keyword argument 'input_value'

In [14]:
from pygromos.files.trajectory.tre import Tre

in_path=project_dir+"/b_vacuum_sd/analysis/data/b_vacuum_sd.tre.h5"

tre = Tre(input_value=in_path)
tre



FileNotFoundError: File /home/runner/work/PyGromosTools/PyGromosTools/examples/example_files/TI_Calculation/M030_to_6KET/b_vacuum_sd/analysis/data/b_vacuum_sd.tre.h5 does not exist

In [15]:
#Plot Potential Energies
V_tot = np.array(list(map(lambda x: x[2], tre.database.totals)))
step = len(tre.database.TIMESTEP_time)//10

plt.plot(tre.database.TIMESTEP_time, V_tot)
plt.xticks(np.round(list(tre.database.TIMESTEP_time[::step]),2))
plt.xlabel("$t~[ps]$")
plt.ylabel("$V~[kJ]$")
plt.title("V total timeseries")
plt.savefig(out_ana+"/potential_energy_timeseries.png")



NameError: name 'tre' is not defined

## Lambda Sampling

### Setup again

In [16]:
import os
import numpy as np
from pygromos.files.gromos_system import Gromos_System
from pygromos.simulations.hpc_queuing.submission_systems.local import LOCAL as subSystem

from pygromos.utils import bash
sys_name = "M030_to_6KET"
root_dir = os.getcwd()+"/example_files/TI_Calculation"

project_dir = bash.make_folder(root_dir+"/"+sys_name)
sys_path = root_dir+"/M030_to_6KET/b_vacuum_sd/sd_out_system.obj"
sd_gromos_system = Gromos_System.load(sys_path)

sd_gromos_system.imd.WRITETRAJ.NTWG =  sd_gromos_system.imd.WRITETRAJ.NTWX = sd_gromos_system.imd.WRITETRAJ.NTWE =10
sd_gromos_system.imd.STEP = 100

sd_gromos_system

mkdir: cannot create directory ‘/home/runner/work/PyGromosTools/PyGromosTools/docs/sphinx_project/Examples/example_files/TI_Calculation/M030_to_6KET’: No such file or directory


OSError: could not make folder:
 /home/runner/work/PyGromosTools/PyGromosTools/docs/sphinx_project/Examples/example_files/TI_Calculation/M030_to_6KET

## Submission

In [17]:
from pygromos.simulations.modules.ti_modules import TI_sampling

step_name  = "d_lambda_sampling"


TI_sampling(in_gromos_system = sd_gromos_system, project_dir=project_dir, step_name  = step_name,
            lambda_values= np.arange(0, 1.1, 0.1),
            subSystem=subSystem(), n_simulation_repetitions = 3, n_equilibrations = 1)


NameError: name 'sd_gromos_system' is not defined